In [97]:
from flask import Flask
from flask import url_for, render_template, request, redirect
import mysql.connector as mysql

app = Flask(__name__)



In [98]:
session = {}
session['username'] = None;

In [99]:
con = mysql.connect(user = 'root', password = 'password', 
                    host = '127.0.0.1', database = 'task_manager')
cur = con.cursor(dictionary=True)

In [100]:
@app.route('/home')
def index():
    if session['username'] == None:
        return render_template('home.html')
    else:
        return render_template('home_logged_in.html')

In [101]:
@app.route('/signup', methods=['GET', 'POST'])
def sign_up():
    if request.method == "POST":
        details = request.form
        username = details['username']
        password = details['password']
        cur.execute("INSERT INTO users_list (username, password) VALUES (%s, %s)", (username, password))
        con.commit()
        cur.close()
        return redirect('login')
    return render_template('sign_up.html')

In [102]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":
        details = request.form
        username = details['username']
        password = details['password']
        session['username'] = username
    return render_template('login.html')

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2019 12:04:56] "GET /signup HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2019 12:05:02] "POST /signup HTTP/1.1" 302 -
127.0.0.1 - - [09/Dec/2019 12:05:02] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2019 12:05:07] "GET /home HTTP/1.1" 200 -


In [58]:
cur.execute("SELECT * FROM users_list")
res = cur.fetchall()
res

[{'id_user': 1, 'password': 'йцукен', 'username': 'fishowl'},
 {'id_user': 2, 'password': 'фывапро', 'username': 'mcqueen'},
 {'id_user': 3, 'password': '65432', 'username': 'ejounya'}]

In [57]:
cur.execute("DELETE FROM users_list WHERE id_user > 3")